In [1]:
import wandb



wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: ed24s401 (ed24s401-indian-institute-of-technology-madras). Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [18]:
!pip install thop -q

In [2]:
from torchvision import datasets, transforms
from torch.utils.data import DataLoader


In [5]:
# transformations for training and testing
transform_train = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5),
                         (0.5, 0.5, 0.5))

])
transform_test = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5),
                         (0.5, 0.5, 0.5))
])



train_dataset = datasets.ImageFolder(root='/kaggle/input/nature-12k/inaturalist_12K/train', transform=transform_train)

test_dataset = datasets.ImageFolder(root='/kaggle/input/nature-12k/inaturalist_12K/val', transform=transform_test)



# Creating Dataloaders

train_loader= DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=2)
train_loader= DataLoader(train_dataset, batch_size=64, shuffle=False, num_workers=2)



11999


In [19]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class FlexibleCNN(nn.Module):
  def __init__(
      self,
      in_channels= 3,
      conv_channels=[32, 64, 128, 256, 512],
      kernel_size= 3,
      activation =nn.ReLU,
      num_classes=10,
      init_image_size=224,
      dense_neurons=512
    ):
    
    super(FlexibleCNN, self).__init__()

    self.activation=activation()

    # Building Conv Layers

    conv_layers =[]
    current_in_channels =in_channels
    current_image_size=init_image_size

    for out_channels in conv_channels:

      # Conv layer
      conv_layers.append(nn.Conv2d(
          in_channels=current_in_channels,
          out_channels=out_channels,
          kernel_size=kernel_size,
          stride=1,
          padding= kernel_size // 2
      

      ))

      # Activation
      conv_layers.append(activation())

      # Max Pooling
      conv_layers.append(nn.MaxPool2d(kernel_size=2, stride=2))

      # Update for next iteration
      current_in_channels = out_channels
      current_image_size = current_image_size // 2

    # Sequential Combination of all conc layers
    self.conv_layers =nn.Sequential(*conv_layers)

    # Flattening
    flattened_dim = current_in_channels * current_image_size**2

    self.fc1 = nn.Linear(flattened_dim, dense_neurons)
    self.fc2 = nn.Linear(dense_neurons, num_classes)


  def forward(self, x):

    # Pass through all conv + pool blocks

    x= self.conv_layers(x)

    # FLatten
    x = x.view(x.size(0), -1)

    # Fully layers with activation
    x=self.activation(self.fc1(x))
    x=self.fc2(x)
    
    return x


model = FlexibleCNN(
    in_channels=3,
    conv_channels=[32, 64, 128, 256, 512],
    kernel_size=3,
    activation=nn.ReLU,
    num_classes=len(train_dataset.classes),  # For iNaturalist or any other dataset
    init_image_size=224,  # If your images are resized to 224x224
    dense_neurons=1024     # Example size for the penultimate layer
)

#print(type(model))
# 
device =torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

print(type(model))

def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

total_params = count_parameters(model)
print("Total trainable parameters:", total_params)

from thop import profile

# Create a dummy input tensor with batch size 1 and appropriate dimensions
dummy_input = torch.randn(1, 3, 224, 224).to(device)
macs, params = profile(model, inputs=(dummy_input,), verbose=False)
print("Total MACs:", macs)


<class '__main__.FlexibleCNN'>
Total trainable parameters: 27269962
Total MACs: 993896448.0


#### QUESTION 2